In [1]:
import os
import pandas as pd
import numpy as np
import sklearn
from selenium import webdriver
import time

In [2]:
df = pd.read_csv('/Users/nanboo/Desktop/CS4700/ML-NBA-Predictor/data/nbagamedataacross/games.csv')
df_teams = pd.read_csv('/Users/nanboo/Desktop/CS4700/ML-NBA-Predictor/data/nbagamedataacross/teams.csv')
df_teams.sort_values(by=['CITY'], inplace=True)
teamIdToNum = {}
yearToNum = {
    2014: 1,
    2015: 2,
    2016: 3,
    2017: 4,
    2018: 5,
    2019: 6,
    2020: 7,
    2021: 8
}
i = 1
for teamID in df_teams['TEAM_ID']:
    teamIdToNum[teamID] = i
    i+=1

df_teams
print(len(df_teams))
teamIdToNum
df["SEASON"].replace(yearToNum, inplace=True)
df["HOME_TEAM_ID"].replace(teamIdToNum, inplace=True)
df["VISITOR_TEAM_ID"].replace(teamIdToNum, inplace=True)
df = df.dropna()
y = df['HOME_TEAM_WINS']
df.reset_index(inplace = True)
df.drop(['index'], inplace = True, axis = 1)

df

30


,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2021-11-17,22100213,Final,4,30,8,1610612766,97.0,0.438,0.500,...,30.0,59.0,1610612764,87.0,0.367,0.813,0.190,23.0,48.0,1
1,2021-11-17,22100214,Final,9,12,8,1610612765,97.0,0.425,0.750,...,16.0,42.0,1610612754,89.0,0.418,0.737,0.243,14.0,43.0,1
2,2021-11-17,22100215,Final,1,2,8,1610612737,110.0,0.506,0.833,...,28.0,40.0,1610612738,99.0,0.440,0.824,0.268,24.0,42.0,1
3,2021-11-17,22100216,Final,3,6,8,1610612751,109.0,0.458,0.840,...,29.0,47.0,1610612739,99.0,0.393,0.857,0.250,20.0,50.0,1
4,2021-11-17,22100217,Final,16,19,8,1610612748,113.0,0.483,0.824,...,29.0,39.0,1610612740,98.0,0.440,0.786,0.286,18.0,38.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24920,2014-10-06,11400007,Final,1,19,1,1610612737,93.0,0.419,0.821,...,24.0,50.0,1610612740,87.0,0.366,0.643,0.375,17.0,43.0,1
24921,2014-10-06,11400004,Final,5,30,1,1610612741,81.0,0.338,0.719,...,18.0,40.0,1610612764,85.0,0.411,0.636,0.267,17.0,47.0,0
24922,2014-10-06,11400005,Final,13,8,1,1610612747,98.0,0.448,0.682,...,29.0,45.0,1610612743,95.0,0.387,0.659,0.500,19.0,43.0,1
24923,2014-10-05,11400002,Final,28,26,1,1610612761,99.0,0.440,0.771,...,21.0,30.0,1610612758,94.0,0.469,0.725,0.385,18.0,45.0,1


In [3]:
#drop Unnecessary data to train preliminarily
from sklearn.model_selection import train_test_split
df.drop(['GAME_DATE_EST','GAME_ID','GAME_STATUS_TEXT','TEAM_ID_home','PTS_home','TEAM_ID_away','PTS_away','HOME_TEAM_WINS'], inplace = True, axis = 1)

X = df

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#print(y_test)

from sklearn.svm import SVC
clf = SVC(C=100,kernel='linear')
clf.fit(X_train,y_train)
predictionssvm = clf.predict(X_test)

#print(predictionssvm)

ytestnp = y_test.to_numpy()
totalsame = np.sum(ytestnp == predictionssvm)
total = np.size(ytestnp)
percent = (totalsame)/total
print(percent)





0.7768119818133191


In [107]:
#random forest
X = df

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

print(y_test)

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clfRF = RandomForestClassifier(random_state=0)
clfRF.fit(X_train,y_train)
predictionsdeep = clfRF.predict(X_test)
print(predictionsdeep)

#checking to see percentage correct
ytestnp = y_test
totalsame = np.sum(ytestnp == predictionsdeep)
total = np.size(ytestnp)
percent = (totalsame)/total
print(percent)
print(clfRF.get_params())

6155     0
18682    1
17225    0
1457     1
7002     1
        ..
20142    0
2168     0
8053     0
6649     1
11361    1
Name: HOME_TEAM_WINS, Length: 7478, dtype: int64
[0 1 1 ... 0 1 1]
0.843808504947847
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}


In [65]:
#neural network
from sklearn.neural_network import MLPClassifier

X = df

#print(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(y_test)

clf = MLPClassifier(max_iter=500, hidden_layer_sizes = (10,)).fit(X_train, y_train)
predictionsneural = clf.predict(X_test)
print(predictionsneural)

#checking to see percentage correct
ytestnp = y_test
totalsame = np.sum(ytestnp == predictionsneural)
total = np.size(ytestnp)
percent = (totalsame)/total
print(percent)

15953    1
11755    0
14398    1
15776    1
20317    0
        ..
10406    1
23895    1
12294    1
8418     1
10324    1
Name: HOME_TEAM_WINS, Length: 7478, dtype: int64
[1 0 1 ... 1 1 1]
0.8073014174913078


In [68]:
#hyper parameter tuning on SVM
X = df

percents = []
C_range = np.logspace(-2, 2, 5)
print(C_range)
gamma_range = np.logspace(-2, 1, 5)
print(gamma_range)


for i in range(np.size(C_range)):
    for j in range(np.size(gamma_range)):
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        
        #training and predictions
        clf = SVC(gamma = gamma_range[j],C=C_range[i],kernel='linear')
        clf.fit(X_train,y_train)
        predictionssvm = clf.predict(X_test)
            
        #checking to see percentage correct
        ytestnp = y_test.to_numpy()
        totalsame = np.sum(ytestnp == predictionssvm)
        total = np.size(ytestnp)
        percent = (totalsame)/total
        percents.append(percent)

maximumper = max(percents)
maxindex = percents.index(maximumper)
print(maxindex)
perss = pd.DataFrame(percents)
perss

[1.e-02 1.e-01 1.e+00 1.e+01 1.e+02]
[ 0.01        0.05623413  0.31622777  1.77827941 10.        ]
10


,0
0,0.614870
1,0.593207
2,0.593875
3,0.599091
4,0.586788
5,0.732281
6,0.587724
7,0.591869
8,0.589329
9,0.586387


In [146]:
#one hot encoding teama
columnName = ['Team'] * 30
teamList = [0] * 30
table = []
for i in range(len(df)):
    homeTeam = df["HOME_TEAM_ID"].iloc[i]
    visTeam = df["VISITOR_TEAM_ID"].iloc[i]
    teamList[homeTeam-1] = 1
    teamList[visTeam-1] = 1
    table.append(teamList)
    teamList = [0] * 30
    
dfn = pd.DataFrame(table, columns = columnName)
#dfn.reset_index()

#df.reset_index(inplace=True)
#normalized_df=(df-df.mean())/df.std()


dftotal = pd.concat([dfn,df], axis=1)



dftotal.drop(['HOME_TEAM_ID','VISITOR_TEAM_ID'], inplace = True, axis = 1)

#dftotal.to_csv(path_or_buf = '/Users/nanboo/Desktop/CS4700/gamesPredClean.csv')

In [147]:
X = dftotal

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#print(y_test)

from sklearn.svm import SVC
clf = SVC(C=100,kernel='linear')
clf.fit(X_train,y_train)
predictionssvm = clf.predict(X_test)

#print(predictionssvm)

ytestnp = y_test.to_numpy()
totalsame = np.sum(ytestnp == predictionssvm)
total = np.size(ytestnp)
percent = (totalsame)/total
print(percent)

0.7630382455201926


In [131]:
#random forest
X = dftotal

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

print(y_test)

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clfRF = RandomForestClassifier(max_depth= 80, max_features= 3, min_samples_leaf= 1, min_samples_split= 2, n_estimators= 1000,random_state=0)
clfRF.fit(X_train,y_train)
predictionsdeep = clfRF.predict(X_test)
print(predictionsdeep)

#checking to see percentage correct
ytestnp = y_test
totalsame = np.sum(ytestnp == predictionsdeep)
total = np.size(ytestnp)
percent = (totalsame)/total
print(percent)

22683    1
4612     1
9362     0
21745    1
12288    1
        ..
4108     0
1276     0
17832    0
24536    0
13743    0
Name: HOME_TEAM_WINS, Length: 7478, dtype: int64
[0 1 0 ... 0 0 0]
0.8376571275742177


In [108]:
#hyper parameter tuning on deep forest via gridSearch
X = dftotal

parameter_space = {
    'max_depth': [80, 90, 100, 110, None],
    'max_features': [2, 3],
    'min_samples_leaf': [1,2,4],
    'min_samples_split': [1,2,8],
    'n_estimators': [100, 200, 300, 1000]
}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

clfRF = RandomForestClassifier(random_state=0)

from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(clfRF, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train, y_train)

# Best parameter set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan 0.83022853 0.83275043
 0.83217719 0.83326634 0.83011384 0.83126027 0.82954074 0.83045779
        nan        nan        nan        nan 0.82988456 0.83057248
 0.82988465 0.83034323 0.82810797 0.83040058 0.83080167 0.8297701
        nan        nan        nan        nan 0.829598   0.82810785
 0.82816506 0.82701872 0.829598   0.82810785 0.82816506 0.82701872
        nan        nan        nan        nan 0.83257852 0.83515794
 0.83613226 0.83676282 0.83447005 0.83573103 0.83429811 0.83544443
        nan        nan        nan        nan 0.83171895 0.83372498
 0.83355289 0.83429806 0.83303723 0.8351579  0.8351005  0.83406878
        nan        nan        nan        nan 0.83091644 0.83103112
 0.83217741 0.83269323 0.83091644 0.83103112 0.83217741 0.83269323
        nan        

Best parameters found:
 {'max_depth': 80, 'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 1000}
nan (+/-nan) for {'max_depth': 80, 'max_features': 2, 'min_samples_leaf': 1, 'min_samples_split': 1, 'n_estimators': 100}
nan (+/-nan) for {'max_depth': 80, 'max_features': 2, 'min_samples_leaf': 1, 'min_samples_split': 1, 'n_estimators': 200}
nan (+/-nan) for {'max_depth': 80, 'max_features': 2, 'min_samples_leaf': 1, 'min_samples_split': 1, 'n_estimators': 300}
nan (+/-nan) for {'max_depth': 80, 'max_features': 2, 'min_samples_leaf': 1, 'min_samples_split': 1, 'n_estimators': 1000}
0.830 (+/-0.006) for {'max_depth': 80, 'max_features': 2, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
0.833 (+/-0.005) for {'max_depth': 80, 'max_features': 2, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
0.832 (+/-0.006) for {'max_depth': 80, 'max_features': 2, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
0.8

In [ ]:
#{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini',
# 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 
# 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 
# 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 
# 'n_jobs': None, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}

In [118]:
X = dftotal

parameter_space = {
    'hidden_layer_sizes': [(20,10)],
    'activation': ['tanh'],
    'solver': ['sgd', 'lbfgs','adam'],
    'alpha': [0.0001],
    'learning_rate': ['constant','adaptive'],
}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(max_iter=500)

from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train, y_train)

# Best parameter set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Best parameters found:
 {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (20, 10), 'learning_rate': 'constant', 'solver': 'lbfgs'}
0.662 (+/-0.010) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (20, 10), 'learning_rate': 'constant', 'solver': 'sgd'}
0.798 (+/-0.012) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (20, 10), 'learning_rate': 'constant', 'solver': 'lbfgs'}
0.762 (+/-0.097) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (20, 10), 'learning_rate': 'constant', 'solver': 'adam'}
0.662 (+/-0.009) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (20, 10), 'learning_rate': 'adaptive', 'solver': 'sgd'}
0.754 (+/-0.032) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (20, 10), 'learning_rate': 'adaptive', 'solver': 'lbfgs'}
0.687 (+/-0.003) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (20, 10), 'learning_rate': 'adaptive', 'solver': 'adam'}
0.659 (+/-0.008)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [127]:
from sklearn.neural_network import MLPClassifier

X = df

#print(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(y_test)

clf = MLPClassifier(max_iter=500, activation='tanh', hidden_layer_sizes= (10,)).fit(X_train, y_train)
predictionsneural = clf.predict(X_test)
print(predictionsneural)

#checking to see percentage correct
ytestnp = y_test
totalsame = np.sum(ytestnp == predictionsneural)
total = np.size(ytestnp)
percent = (totalsame)/total
print(percent)

21236    0
12247    0
4117     0
17782    0
9109     1
        ..
6785     0
21707    1
8833     0
3137     0
17404    0
Name: HOME_TEAM_WINS, Length: 7478, dtype: int64
[0 1 1 ... 1 1 1]
0.686279753944905


In [152]:
sixChar = [0]*30
sixChar[22] = 1
sixChar[3] = 1
otherList = [8,.459,.741,.384,26.5,44.1,.459,.809,.360,22.8,42.8]
finalList = sixChar+otherList
arr = np.array(finalList)
arr = arr.reshape(1, -1)
clfRF.fit(X,y)
lolisee = clfRF.predict(arr)
predprob = clfRF.predict_proba(arr)
print(lolisee)
print(predprob)

[1]
[[0.373 0.627]]


In [151]:
wizPace = [0]*30
wizPace[11] = 1
wizPace[29] = 1
otherList = [8,.458,.776,.328,23.7,45.7,.462,.782,.325,23.6,45.2]
finalList = wizPace+otherList
arr = np.array(finalList)
arr = arr.reshape(1, -1)
clfRF.fit(X,y)
lolisee = clfRF.predict(arr)
predprob = clfRF.predict_proba(arr)
print(lolisee)
print(predprob)

[1]
[[0.418 0.582]]


In [149]:
knickSpur = [0]*30
knickSpur[19] = 1
knickSpur[26] = 1
otherList = [8,.465,.700,.348,27.7,44.6,.443,.776,.366,21.6,45.7]
finalList = knickSpur+otherList
arr = np.array(finalList)
arr = arr.reshape(1, -1)
clfRF.fit(X,y)
lolisee = clfRF.predict(arr)
predprob = clfRF.predict_proba(arr)
print(lolisee)
print(predprob)

[1]
[[0.3 0.7]]


In [150]:
lakCelt = [0]*30
lakCelt[13] = 1
lakCelt[1] = 1
otherList = [8,.465,.736,.347,24.5,46.4,.443,.819,.338,22.8,46]
finalList = lakCelt+otherList
arr = np.array(finalList)
arr = arr.reshape(1, -1)
clfRF.fit(X,y)
lolisee = clfRF.predict(arr)
predprob = clfRF.predict_proba(arr)
print(lolisee)
print(predprob)

[1]
[[0.265 0.735]]


In [153]:
netMav = [0]*30
netMav[6] = 1
netMav[2] = 1
otherList = [8,.441,.729,.332,22.4,43,.467,.806,.365,25.6,44.9]
finalList = netMav+otherList
arr = np.array(finalList)
arr = arr.reshape(1, -1)
clfRF.fit(X,y)
lolisee = clfRF.predict(arr)
predprob = clfRF.predict_proba(arr)
print(lolisee)
print(predprob)

[0]
[[0.759 0.241]]
